In [2]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import warnings
import os.path as osp
import numpy as np
from general_tools.in_out.basics import create_dir
from tf_lab.evaluate.generative_pc_nets import minimum_mathing_distance
from tf_lab.nips.helper import pclouds_with_zero_mean_in_unit_sphere, center_pclouds_in_unit_sphere

In [5]:
n_pc_samples = 2048
batch_size = 6000
reduce_gt = None        # (int or None)
reduce_samples = None   # (int or None)

class_name = 'chair'
save_res = False
ae_loss = 'emd'

def identity(x):
    return x

pc_normalizer = center_pclouds_in_unit_sphere
pc_normalizer = pclouds_with_zero_mean_in_unit_sphere
pc_normalizer = identity

In [7]:
def sampling_mmd(sample_data, ref_data, n_samples=10, batch_size=100, emd=True):
    ids = np.arange(len(ref_data))
    scores = []
    for i in range(n_samples):
        print i
        sb = np.random.choice(ids, batch_size, replace=False)        
        mi = minimum_mathing_distance(sample_data[sb], ref_data[sb], batch_size, normalize=True, use_sqrt=True, use_EMD=emd)
        scores.append(mi[0])
    scores = np.array(scores)
    return scores

In [16]:
is_lwgan = False
is_rgan = False
is_lgan = True

assert(np.sum([is_lwgan, is_rgan, is_lgan]) == 1)


if is_lwgan:
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(100, 2001, 100)])
    gan_tag = 'l_wgan_gp'
elif is_rgan:
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(50, 2001, 50)])
    gan_tag = 'r_gan'
elif is_lgan:
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(100, 2001, 100)])
    gan_tag = 'l_gan'

epochs_to_check = [1000]

In [9]:
# Load Ground-Truth Train Data.
top_gt_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/gt_data/'
gt_train_file = osp.join(top_gt_dir, class_name + '_train.npz')
gt_train_data = np.load(gt_train_file)  
gt_train_data = pc_normalizer(gt_train_data[gt_train_data.keys()[0]])

# Load Ground-Truth Test Data.
gt_test_file = osp.join(top_gt_dir, class_name + '_test.npz')
gt_test_data = np.load(gt_test_file)  
gt_test_data = pc_normalizer(gt_test_data[gt_test_data.keys()[0]])

if reduce_gt is not None:    
    gt_train_data = gt_train_data[:min(reduce_gt, len(gt_train_data))]
    gt_test_data = gt_test_data[:min(reduce_gt, len(gt_test_data))]

In [17]:
top_in_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/'

if is_lwgan:
    top_synthetic_dir = osp.join(top_in_dir, 'l_w_gan_chair_disc_512_1024_emd_bneck_128_this_was_also_using_3pc_ae/lam_10/')
    special_tag = 'chair_disc_512_1024_emd_bneck_128_lam_10'
elif is_rgan:
    top_synthetic_dir = osp.join(top_in_dir, 'r_gan/chair_mlp_disc_4_fc_gen_raw_gan_2048_pts/')
    special_tag = 'chair_mlp_disc_4_fc_gen_raw_gan_2048_pts'
elif is_lgan:        
    top_synthetic_dir = osp.join(top_in_dir, 'l_gan/l_gan_chair_mlp_with_split_1pc_usampled_bnorm_on_encoder_only_'+ae_loss+'_bneck_128')
    special_tag = 'l_gan_chair_mlp_with_split_1pc_usampled_bnorm_on_encoder_only_emd_bneck_128'

In [18]:
top_out_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/mmd'
create_dir(top_out_dir)

'/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/mmd'

In [19]:
if save_res:
    out_file = '_'.join([gan_tag, special_tag, 'norm', pc_normalizer.__name__])
    out_file = osp.join(top_out_dir, out_file + '.txt')
    fout = open(out_file, 'w', 1)
    fout.write('#Metric Epoch (Train-Test) Measurements\n')
    print 'Saving measurements at: ' + out_file

for epoch in epochs_to_check:
    sample_file = osp.join(top_synthetic_dir, 'epoch_%d.npz' % (epoch,) )
    sample_data = np.load(sample_file)
    sample_data = sample_data[sample_data.keys()[0]]
    sample_data = pc_normalizer(sample_data)
    mmd_train = sampling_mmd(sample_data, gt_train_data, n_samples=10, batch_size=200, emd=True)
    
#     if reduce_samples is not None:
#         sample_data = sample_data[:reduce_samples]
        
#     mmd_train = minimum_mathing_distance(sample_data, gt_train_data, batch_size, normalize=True, use_sqrt=True, verbose=True)[0]
#     mmd_test = mmd_train
    
# #     minimum_mathing_distance(sample_data[:len(gt_test_data)], gt_test_data, batch_size, normalize=True, use_sqrt=True)[0]
    
#     log_data = 'MMD %d %f %f' % (epoch, mmd_train, mmd_test)
#     print log_data
#     if save_res:
#         fout.write(log_data + '\n')

# if save_res:
#     fout.close()

0
1
2
3
4
5
6
7
8
9


In [ ]:
For l-gan: 263.732 5.58732 at 1K epoch - train

In [ ]:
For r-gan: 348.764 2.77347 at 1K epoch - train